In [100]:
import sys
sys.path.append("..")
from lib.util import add_race_eth,  add_income_group
import pandas as pd
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1550
import numpy as np
from ambry import library
l = library()
p = l.get('ffiec.gov-hmda-2010e-lar-2012-ca').partition
df = p.select("SELECT * FROM lar WHERE county_code = 73").pandas

In [101]:
add_race_eth(df)
dfs = df[['id','census_tract_number', 'applicant_ethnicity','applicant_race_1', 'race_eth']]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 242783 entries, 0 to 242782
Data columns (total 47 columns):
id                                242783 non-null int64
year                              242783 non-null int64
respondent_id                     242783 non-null object
agency_code                       242783 non-null int64
loan_type                         242783 non-null int64
property_type                     242783 non-null int64
loan_purpose                      242783 non-null int64
occupancy                         242783 non-null int64
loan_amount                       242783 non-null int64
preapproval                       242783 non-null int64
action_type                       242783 non-null int64
msa_md                            242706 non-null float64
state_code                        242783 non-null int64
county_code                       242783 non-null int64
census_tract_number               242783 non-null object
applicant_ethnicity               242783 non-

## Total Applications (1)
- **Missing: Minority Status, Tract/BNA**

In [145]:
totalcount = df.loan_amount.count()
totalamount = df.loan_amount.sum()
print("total number of loans",totalcount)
print("total amount of loans",totalamount)

('total number of loans', 242783)
('total amount of loans', 82066044)


### By Loan Purpose and Type

In [146]:
# classification
df['loan_purtype'] = '_'
df['loan_purtype'][(df['loan_purpose'] == 1) & (df['loan_type'] == 1)] = 'Purchase - Conventional'
df['loan_purtype'][(df['loan_purpose'] == 1) & (df['loan_type'] > 1)] = 'Purchase - Goverment'
df['loan_purtype'][df['loan_purpose'] == 2] = 'Home Improvement'
df['loan_purtype'][df['loan_purpose'] == 3] = 'Refinancing'

loanpt = df.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/totalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/totalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

### By Applicant Race/Ethnicity

In [147]:
race_eth  = df.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/totalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/totalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

###By Applicant Income

In [148]:
df.applicant_income.median()

104.0

In [149]:
# classification of applicant income as percent of median
df['applicant_income_percent_median'] = 100*df.applicant_income/df.applicant_income.median()
df['applicant_income_cat'] = 'Income Not Available'
df['applicant_income_cat'][df['applicant_income_percent_median'] < 50] = 'Low'
df['applicant_income_cat'][(df['applicant_income_percent_median'] >= 50) & (df['applicant_income_percent_median'] < 80)] = 'Moderate'
df['applicant_income_cat'][(df['applicant_income_percent_median'] >= 80) & (df['applicant_income_percent_median'] < 120)] = 'Middle'
df['applicant_income_cat'][df['applicant_income_percent_median'] >= 120] = 'Upper'

income  = df.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/totalcount,1)
income ['%amount'] = np.round(100*income['amount']/totalamount,1)
income = income [['count','%count','amount','%amount']]

###By Applicant Sex

In [150]:
sex  = df.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/totalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/totalamount,1)
sex = sex [['count','%count','amount','%amount']]

In [152]:
frames = [loanpt, race_eth,income,sex]
totalapp = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
totalapp
pieces = [totalapp.ix[:, ["count","%count","amount","%amount",]]]
totalapp = pd.concat(pieces, axis=1, keys=['Total Applications'])
totalapp

Total Applications          \
                                                            count  %count   
Loan Purpose and Type Home Improvement                       4781     2.0   
                      Purchase - Conventional               32857    13.5   
                      Purchase - Goverment                  22240     9.2   
                      Refinancing                          182905    75.3   
Applicant Race/Eth    aian                                    767     0.3   
                      as                                    23920     9.9   
                      b                                      4053     1.7   
                      h                                     24012     9.9   
                      pi                                     1908     0.8   
                      w                                    142000    58.5   
Applicant Income      Income Not Available                  28473    11.7   
                      Low                                   26217    10.8   
                      Middle                                57469    23.7   
                      Moderate                              49857    20.5   
                      Upper                                 80767    33.3   
Applicant Sex         1                                    159485    65.7   
                      2                                     50394    20.8   
                      3                                     17527     7.2   
                      4                                     15377     6.3   

                                                                  
                                                 amount  %amount  
Loan Purpose and Type Home Improvement          1038687      1.3  
                      Purchase - Conventional  12419787     15.1  
                      Purchase - Goverment      7551607      9.2  
                      Refinancing              61055963     74.4  
Applicant Race/Eth    aian                       231319      0.3  
                      as                        7808190      9.5  
                      b                         1139822      1.4  
                      h                         6385159      7.8  
                      pi                         570463      0.7  
                      w                        48384565     59.0  
Applicant Income      Income Not Available     11876933     14.5  
                      Low                       4777981      5.8  
                      Middle                   17701587     21.6  
                      Moderate                 12375557     15.1  
                      Upper                    35333986     43.1  
Applicant Sex         1                        54930806     66.9  
                      2                        14172171     17.3  
                      3                         5939329      7.2  
                      4                         7023738      8.6

#By Bank

In [116]:
ts = l.get('ffiec.gov-hmda-2010e-ts').partition.pandas
ts_name = ts[['respondent_id','respondent_name']]
x = source[['id','respondent_id', 'loan_type']].merge(ts_name, on='respondent_id')
x[x.respondent_name.str.contains('BANK OF AMERICA')][['respondent_id','respondent_name']].drop_duplicates()

NameError: name 'source' is not defined

### Bank of America: 
- respondent_id = 0000480228 

In [154]:
boatotalcount = boadf.loan_amount.count()
boatotalamount = boadf.loan_amount.sum()

# classification
boadf['loan_purtype'] = '_'
boadf['loan_purtype'][(boadf['loan_purpose'] == 1) & (boadf['loan_type'] == 1)] = 'Purchase - Conventional'
boadf['loan_purtype'][(boadf['loan_purpose'] == 1) & (boadf['loan_type'] > 1)] = 'Purchase - Goverment'
boadf['loan_purtype'][boadf['loan_purpose'] == 2] = 'Home Improvement'
boadf['loan_purtype'][boadf['loan_purpose'] == 3] = 'Refinancing'

loanpt = boadf.groupby(['loan_purtype'])
loanpt = loanpt['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})
loanpt['%count'] = np.round(100*loanpt['count']/boatotalcount,1)
loanpt['%amount'] = np.round(100*loanpt['amount']/boatotalamount,1)
loanpt=loanpt[['count','%count','amount','%amount']]

race_eth  = boadf.groupby(['race_eth'])
race_eth  = race_eth ['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

race_eth ['%count'] = np.round(100*race_eth['count']/boatotalcount,1)
race_eth ['%amount'] = np.round(100*race_eth['amount']/boatotalamount,1)
race_eth = race_eth [['count','%count','amount','%amount']]

# classification of applicant income as percent of median
boadf['applicant_income_percent_median'] = 100*boadf.applicant_income/boadf.applicant_income.median()
boadf['applicant_income_cat'] = 'Income Not Available'
boadf['applicant_income_cat'][boadf['applicant_income_percent_median'] < 50] = 'Low'
boadf['applicant_income_cat'][(boadf['applicant_income_percent_median'] >= 50) & (boadf['applicant_income_percent_median'] < 80)] = 'Moderate'
boadf['applicant_income_cat'][(boadf['applicant_income_percent_median'] >= 80) & (boadf['applicant_income_percent_median'] < 120)] = 'Middle'
boadf['applicant_income_cat'][boadf['applicant_income_percent_median'] >= 120] = 'Upper'

income  = boadf.groupby(['applicant_income_cat'])
income   = income['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

income ['%count'] = np.round(100*income['count']/boatotalcount,1)
income ['%amount'] = np.round(100*income['amount']/boatotalamount,1)
income = income [['count','%count','amount','%amount']]

sex  = boadf.groupby(['applicant_sex'])
sex   = sex['loan_amount'].agg(
                            {'count': np.size,
                             'amount': np.sum})

sex ['%count'] = np.round(100*sex['count']/boatotalcount,1)
sex ['%amount'] = np.round(100*sex['amount']/boatotalamount,1)
sex = sex [['count','%count','amount','%amount']]

frames = [loanpt, race_eth,income,sex]
boaapp = pd.concat(frames, keys=['Loan Purpose and Type', 
                                   'Applicant Race/Eth', 
                                   'Applicant Income',
                                   'Applicant Sex'])
pieces = [boaapp.ix[:, ["count","%count","amount","%amount",]]]
boaapp = pd.concat(pieces, axis=1, keys=['Bank of America'])

/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:29: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead
/Library/Python/2.7/site-packages/IPython/kernel/__main__.py:30: SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index,col_indexer] = value instead


In [155]:
#merging
pd.merge(totalapp, boaapp, left_index=True, right_index=True)

Total Applications          \
                                                            count  %count   
Loan Purpose and Type Home Improvement                       4781     2.0   
                      Purchase - Conventional               32857    13.5   
                      Purchase - Goverment                  22240     9.2   
                      Refinancing                          182905    75.3   
Applicant Race/Eth    aian                                    767     0.3   
                      as                                    23920     9.9   
                      b                                      4053     1.7   
                      h                                     24012     9.9   
                      pi                                     1908     0.8   
                      w                                    142000    58.5   
Applicant Income      Income Not Available                  28473    11.7   
                      Low                                   26217    10.8   
                      Middle                                57469    23.7   
                      Moderate                              49857    20.5   
                      Upper                                 80767    33.3   
Applicant Sex         1                                    159485    65.7   
                      2                                     50394    20.8   
                      3                                     17527     7.2   
                      4                                     15377     6.3   

                                                                  \
                                                 amount  %amount   
Loan Purpose and Type Home Improvement          1038687      1.3   
                      Purchase - Conventional  12419787     15.1   
                      Purchase - Goverment      7551607      9.2   
                      Refinancing              61055963     74.4   
Applicant Race/Eth    aian                       231319      0.3   
                      as                        7808190      9.5   
                      b                         1139822      1.4   
                      h                         6385159      7.8   
                      pi                         570463      0.7   
                      w                        48384565     59.0   
Applicant Income      Income Not Available     11876933     14.5   
                      Low                       4777981      5.8   
                      Middle                   17701587     21.6   
                      Moderate                 12375557     15.1   
                      Upper                    35333986     43.1   
Applicant Sex         1                        54930806     66.9   
                      2                        14172171     17.3   
                      3                         5939329      7.2   
                      4                         7023738      8.6   

                                               Bank of America          \
                                                         count  %count   
Loan Purpose and Type Home Improvement                     103     1.0   
                      Purchase - Conventional             1022    10.3   
                      Purchase - Goverment                 550     5.5   
                      Refinancing                         8271    83.2   
Applicant Race/Eth    aian                                  40     0.4   
                      as                                  1023    10.3   
                      b                                    201     2.0   
                      h                                   1463    14.7   
                      pi                                    81     0.8   
                      w                                   5668    57.0   
Applicant Income      Income Not Available                1462    14.7   
                      Low             

##Single-figure statistics
##median loan size 
- countywide
- by census tract
- by geographic area (see below)
- by demographic
	- low-income
	- moderate-income
	- low AND moderate-income
	- middle income
	- high income
	- male, female
	- race/ethnicity 

In [110]:
#median by countywide
df.county_code.unique()

array([73])

In [111]:
df.loan_amount.describe()

count    242783.000000
mean        338.022201
std         488.944942
min           1.000000
25%         210.000000
50%         299.000000
75%         400.000000
max       99500.000000
dtype: float64

In [112]:
#median by census tract
censustract = df.groupby('census_tract_number').median()
censustract[['loan_amount', 'year' ]]

,loan_amount,year
census_tract_number,,
0001.00,442.5,2012
0002.01,411.5,2012
0002.02,387.0,2012
0003.00,326.5,2012
0004.00,325.5,2012
0005.00,361.0,2012
0006.00,305.5,2012
0007.00,282.0,2012
0008.00,289.0,2012


In [113]:
#median by gender
gender = df.groupby('applicant_sex').median()
gender[['loan_amount', 'year' ]]

,loan_amount,year
applicant_sex,,
1,312,2012
2,258,2012
3,295,2012
4,300,2012
